#  TESTA BERT

Attribution: Chris McCormick and Nick Ryan. (2019, May 14). BERT Word Embeddings Tutorial. Retrieved from http://www.mccormickml.com

## 1. Hämta en BERT modell

In [2]:
#Ladda en förtränad BERT-modell
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Formatera input 

### Testa BERTS tokenizerare för att formatera input till korrekt format för BERT

BERT kan ta en eller två meningar som input

Först kommer alltid [CLS] och mellan varje mening kommer [SEP]

Att annotera meningar med [SEP] kan göras automatiskt. Alla meningar måste ha samma längd, även detta kan ordnas automatiskt, men man kan behöva kontrollera att det blivit som man tänkt sig. 

In [3]:
text = "Skriv en mening här, vad som helst."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]']


In [ ]:
#Lista lite ord som finns i base BERT:s vokabulär
list(tokenizer.vocab.keys())[5000:5020]

In [4]:
# Skriv en ny mening där du inkluderar ett ord som har olika betydelse
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Lägg till de viktiga specialtokens som BERT kräver.
marked_text = "[CLS] " + text + " [SEP]"

# Dela upp meningarna i tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Mappa tokens till index i vokabuläret 
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Visa orden med deras index 
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


### Segment ID

BERT förväntar sig parvisa meningar där det är tydligt vilka ord som för till vilken mening. 

In [7]:
# Vi har bara en mening, alltså tillhör alla våra tokens mening 1
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Extrahera embeddings

Vi använder oss här av Torch bilioteket vilken förväntar sig tensors, inte listor eller tuples. Därför måste vi skapa tensorer, det förändrar inte själva data

In [8]:
# Konvertera input till PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

### Kör BERT med vår mening

In [9]:
# Ladda den förtränade modellen (vikterna för modellen)  
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Om modellen ska returnera alla hidden states dvs alla vektorer som skapats
                                  )

# Feed forward mode, kallas för evaluation mode. Det finns flera andra för att utföra andra uppgifter 
model.eval()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
# Kör din text genom BERT. och samla alla hidden states som producerats från alla 12 lager i BERT. no_grad nedan innebär att vi inte räknar ut hur det gick, 
# eftersom vi inte ska bakåtpropagera, eftersom vi inte tränar, sparar minne.  
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # När vi utvärderar modellen kommer den returnera ett antal objekt, beroende på hur den är konfigurerad. Eftersom vi satt output_hidden_state = True
    # Så kommer det tredje objektet att vara hidden state från alla lager. Se dokumentationen för detlajer: https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [11]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 22
Number of hidden units: 768


Alla hidden states för modellen är nu lagrade i objektet hidden_states. Objektet har däremd 4 dimensioner.  

* The layer number (13 layers)
* The batch number (1 sentence)
* The word / token number (Hur många tokens i din mening?)
* The hidden unit / feature number (768 features)

Varför 13 lager? BERT base har ju 12? Det första elementet är input embeddingen, resten är output från varje lager av de 12 lager som BERT har. 
Det innebär att vår mening har ?? unika värden bara för en enda mening! 

Den andra dimensionen, batch size, används när man ska ge modellen flera meningar samtidigt. 


# So what?¨

För varje token vi matat in i BERT har den spottat ur sig 13 separata vectorer, varje har en längd på 768 

Vi har alltså en mängd gigantiska vektorer som representerar vår enda mening vi matat in. Vad ska vi göra nu? 


## Skapa vektorer för ord och meningar 

Från alla dessa vektorer så kan vi skapa vektorer för varje enskilt ord eller för en hel mening genom att kombinera vektorer från olika lager. Vi vet dock inte exakt vilka kombinationer som är "bäst" för att representera vår mening. 

In [13]:
# Konkatenera tensorerna för alla lager och skapa en ny dimension i tensorn. 
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 22, 768])

In [14]:
# Ta bort dimentionen som representerar batches eftersom vi bara har en mening.
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 22, 768])

In [16]:
# Byt plats på dimension 0 och 1 för att få tokens först i tensorn (grupperad på tokens)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([22, 13, 768])

### Ordvektorer

Finns som sagt flera sätt att skapa vektorer, ett sätt att att konkatenera de sista fyra lagren. Ett annat att summera de sista fyra lagren. Båda är kodade nedan. 

In [17]:
# Spara token vektorer, med storlek [22 * 3 072]
token_vecs_cat = []

# `Embeddings för tokens är en tensor med dimensionerna [22 x 12 x 768] 

# För varje token i vår mening...
for token in token_embeddings:
    
    # `token` är en [12 x 768] tensor

    # Conkatenera vektorerna
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Använd `cat_vec` för att representera `token`.
    token_vecs_cat.append(cat_vec)

print ('Storleken är: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Storleken är: 22 x 3072


In [18]:
# Spara tokenvectorer med storlek [22 x 768]
token_vecs_sum = []

# `token_embeddings` är en [22 x 12 x 768] tensor.


for token in token_embeddings:
    #Summera de fyra sista lagren
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)

print ('Storleken är: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Storleken är: 22 x 768


### Vektor för meningar

Återigen finns flera olika sätt, vilket som är bäst beror på vad man vill göra. Enkelt sätt är att ta medelvärdet för alla lager från lager 2 till sista lagret. Då blir resultatet en vektor med längd 768.

In [21]:
# `hidden_states` har storleken [13 x 1 x 22 x 768]

# `token_vecs` är en tensor med storleken [22 x 768]
token_vecs = hidden_states[-2][0]

# Beräkna medelvärdet för alla 22 tokens
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


### Har samma ord betydelse som beror på kontexten (orden omkring)? 

In [22]:
# Vilka index finns vårt ord med flera betydelser på? 
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 after
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 robber
12 was
13 seen
14 fishing
15 on
16 the
17 mississippi
18 river
19 bank
20 .
21 [SEP]


Index 6, 10, och 19.


In [24]:
print('Första 5 vektorvärdena för varje instans av ordet')
print('')
print("bank vault   ", str(token_vecs_sum[6][:5]))
print("bank robber  ", str(token_vecs_sum[10][:5]))
print("river bank   ", str(token_vecs_sum[19][:5]))

Första 5 vektorvärdena för varje instans av ordet

bank vault    tensor([ 3.3596, -2.9805, -1.5421,  0.7065,  2.0031])
bank robber   tensor([ 2.7359, -2.5577, -1.3094,  0.6797,  1.6633])
river bank    tensor([ 1.5266, -0.8895, -0.5152, -0.9298,  2.8334])


Vi kan se att de är olika, men för säkerhetsskull kan vi beräkna cosinuslikheten

In [27]:
from scipy.spatial.distance import cosine

# Beräkna cosinuslikehten mellan ordets olika betydelse 
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Beräkna för ordet i lika betydelse
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Likhet för *liknande*  betydelse:  %.2f' % same_bank)
print('Likhet för *olik* betydelse:  %.2f' % diff_bank)

Likhet för *liknande*  betydelse:  0.94
Likhet för *olik* betydelse:  0.69
